<a href="https://colab.research.google.com/github/TALeonard/19ma573thomasleonard/blob/master/src/hw6_exact_sample_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we will import modules.

In [1]:
#!git clone https://github.com/TALeonard/18ma573pub.git #Only run once!

Cloning into '18ma573pub'...
remote: Enumerating objects: 763, done.
remote: Total 763 (delta 0), reused 0 (delta 0), pack-reused 763
Receiving objects: 100% (763/763), 2.47 MiB | 25.57 MiB/s, done.
Resolving deltas: 100% (449/449), done.


In [2]:
cd /content/18ma573pub/src/

/content/18ma573pub/src


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from contract_v01 import VanillaOption
from sde_1d_v01 import Sde_1d, Gbm_1d

As I am only using it this one time (for the moment, at least), I will simply re-run my Asian Option code from HW4. I acknowledge that I should probably make an updated form of either sde_1d_v01 or contract_v01 at some point but I do not need to do so quite yet.

In [0]:
'''==============
output: BSM geometric asian option price
==============='''

def bsm_geometric_asian_price(self,
                             otype = 1,
                             strike = 110,
                             maturity = 1,
                             num_step = 4 #partition number
                             ):
  s0 = self.init_state
  sigma = self.vol_ratio
  r = self.drift_ratio
  n = num_step
  
  #NOTE: This price assumes a uniform partition time steps, for ease of coding.
  
  #Want to compute r-hat and sigma-hat for the price itself.
  #r-hat = mu-hat + 1/2(sigma-hat^2)
  #Thus for r-hat we need mu-hat, and sigma-hat
  
  #Compute mu-hat
  #mu-hat = mu/2 (we are assuming uniform partition as stated above)
  mu = r - 0.5*(sigma**2)
  mu_hat = (mu/2)
  
  #Compute sigma-hat
  #Sigma-hat^2 = (sigma^2 * (2m + 1))/(6*(m+1)) where m is the number of steps.
  #Recall that the vol_ratio is sigma, not sigma^2. Thus we need both.
  sigma_hat_squared = ((sigma**2)*((2*n) + 1))/(6*(n+1))
  sigma_hat = sigma_hat_squared ** 0.5
  
  #With mu-hat and sigma-hat calculated, we can find r-hat.
  r_hat = mu_hat + (0.5 * sigma_hat_squared)
  
  #Create a separate GBM variable to store the new sigma and r, for the sake of
  #computing the price.
  
  gao_internal_gbm = Gbm_1d(init_state=s0,drift_ratio = r_hat,vol_ratio = sigma_hat)
  
  return np.exp((r_hat - r)*maturity) * gao_internal_gbm.bsm_price(VanillaOption(otype,strike,maturity))

Gbm_1d.bsm_geometric_asian_price = bsm_geometric_asian_price

In [0]:
'''=================
paras are given here
================='''

s0 = 100.0
sigma = 0.20
r = 0.0475

K = 110.0
T = 1.
otype = 1
num_step = 5

Now, create arithmetic asian exact sample code. First, make code to produce our BM paths.

In [0]:
def BM_gen(T1, T2, n): #para: start time, end time, mesh numbr
  t = np.linspace(T1, T2, num = n+1)
  W = np.zeros(n+1)
  #Run (3.2)
  for i in range(n):
    W[i+1] = W[i] + 1./np.sqrt(n) * np.random.normal()
    
  return t, W

In [0]:
def bsm_arithmetic_asian_exact_sample(self,
                                     otype = 1,
                                     strike = 110,
                                     maturity = 1,
                                     num_step = 5,
                                     num_path = 100):
  s0 = self.init_state
  sigma = self.vol_ratio
  r = self.drift_ratio
  n = num_step
  m = num_path
  
  Stock_history = np.zeros((num_path,n+1))
  Stock_history[:,0] = s0;
  
  A = np.zeros(num_path)
  C_sample = np.zeros(num_path)
  
  
  ### UNCERTAINTY: HOW TO SET UP TIME STEP SYSTEM
  
  # Generate (num_path) many GBM paths by exact sampling.
  
  for i in range(m):
    [t,W] = BM_gen(0,maturity,n)
    for j in range(n):
      Stock_history[i,j+1] = s0 * np.exp((r-(0.5*(sigma**2)))*t[j+1] + sigma*W[j+1])
    # Compute discounted payoff for each path.
    A[i] = np.mean(Stock_history[i,:])
    if A[i] - strike > 0:
      C_sample[i] = np.exp(-1*r*maturity) * (A[i]-strike)
    else:
      C_sample[i] = 0

  # Take average to find option price.
  return np.mean(C_sample)

Gbm_1d.bsm_arithmetic_asian_exact_sample = bsm_arithmetic_asian_exact_sample

Now, run to find the price of the three calls for comparison. Given inherent randomness in price of the Arithmetic, have 2 versions run (to show that the outputs will differ despite having the exact same inputs).

In [8]:
gbm1 = Gbm_1d(init_state = s0, drift_ratio = r, vol_ratio = sigma)

eur_call1 = gbm1.bsm_price(VanillaOption(otype = otype,
                                     strike = K,
                                     maturity = T,
                                     market_price = s0
                                       ))

gao1 = gbm1.bsm_geometric_asian_price(
                          otype = otype,
                          strike = K,
                          maturity = T,
                          num_step = num_step
                          )
aac1 = gbm1.bsm_arithmetic_asian_exact_sample(
                                        otype=otype,
                                        strike=K,
                                        maturity=T,
                                        num_step=num_step,
                                        num_path=500
                                        )
aac2 = gbm1.bsm_arithmetic_asian_exact_sample(
                                        otype=otype,
                                        strike=K,
                                        maturity=T,
                                        num_step=num_step,
                                        num_path=500
                                        )

print('European Call Option Price is ' + str(eur_call1))
print('Geometric Asian Call Option Price is ' + str(gao1))
print('First Arithmetic Asian Call Option Price is ' + str(aac1))
print('Second Arithmetic Asian Call Option Price is ' + str(aac2))

European Call Option Price is 5.943273183452838
Geometric Asian Call Option Price is 1.6421227433993888
First Arithmetic Asian Call Option Price is 2.1514044384703666
Second Arithmetic Asian Call Option Price is 2.0216902763622078


We can see that the prices of both "runs" of the Arithmetic option are larger than the Geometric option here. If we keep running, we can occasionally see that one or both of these will dip below the Geometric option. The frequency with which this occurs goes down if we increase the number of paths.

Despite this, I believe that we can make the following comparison overall, with respect to options pricing:

Geometric Asian Call $\leq$ Arithmetic Asian Call $\leq$ European Call

This is a combination of the following arguments:

* The European Call's price is relatively much larger than the other 2 options prices, which are roughly equal to each other. Thus, we can identify it as the largest of the three.

* The Geometric option's price here is fixed at $\$1.64$, whereas the Arithmetic option's price can go above (or below) that. Since the Arithmetic can be more expensive, I believe it makes the most sense to say the Arithmetic is the more expensive of the two. This is backed up by the fact that generally speaking, the Geometric Mean of a set will always be less than or equal to its Arithmetic Mean. This is a basic property of the [Pythagorean Means](https://en.wikipedia.org/wiki/Pythagorean_means), of which Arithmetic and Geometric are 2 of the 3 (the third being the Harmonic mean).

For completeness, let us attempt to identify the "average" of these averaged prices (using the arithmetic mean), to verify this.

In [11]:
sum_price = 0
n = 100

for i in range(n):
  aac_out = gbm1.bsm_arithmetic_asian_exact_sample(
                                        otype=otype,
                                        strike=K,
                                        maturity=T,
                                        num_step=num_step,
                                        num_path=500
                                        )
  sum_price += aac_out

mean_price = sum_price/n
print("Average of 'average' Arithmetic Asian Call Prices is " + str(mean_price))

Average of 'average' Arithmetic Asian Call Prices is 1.8600676411637982


We can see that the "average" asian call prices are averaging out to something between $\$1.80$ and $\$1.90$. We could increase the num_path variable or the number of samples $n$ used in the above code to get a more precise result, at the cost of a higher time spent computing (more paths to simulate and average will increase the time the code takes to run).

Based on this, therefore, I am confident in my original remark that the Arithmetic Asian Call's price is larger than the Geometric Asian Call's price.